In [1]:
# We install the package through devtools::install_github
# This requires the "devtools" package to be installed
install.packages("devtools",  INSTALL_opts = c('--no-lock'))
library(devtools)
devtools::install_github("TobiasRoikjer/PtDAlgorithms")

Installing package into ‘/home/tobias/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

Loading required package: usethis





✔  checking for file ‘/tmp/RtmpThPsM1/remotes53755078a75/TobiasRoikjer-PtDAlgorithms-a4fbecd/DESCRIPTION’ (408ms)
─  preparing ‘ptdalgorithms’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts (426ms)
─  checking for empty or unneeded directories
   Omitted ‘LazyData’ from DESCRIPTION
─  building ‘ptdalgorithms_1.0.0.tar.gz’
   


Installing package into ‘/home/tobias/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



In [2]:
# Import the functions in the library
library(ptdalgorithms)

# The library supports setting seeds for some of its
# sampling capability, as any other R package
set.seed(1234)

In [3]:
# While the R api can create graphs, this is *slow* as the C++ binding layer
# through Rcpp is slow when invoking many functions, and since R is a slow,
# interpreted language
# Clone or download the code, and include these files in the repository!
# Make SURE that the version of the downloaded code is the same as the
# installed R library!! Otherwise it may crash randomly
# The file has comments and is easy to understand, so you should be able
# to defined you own cool construction functions

Rcpp::sourceCpp("./isolation_migration_model.cpp")

In [4]:
# Construction of IM model and computation of expectations

n1 <- 8
n2 <- 8

time <- proc.time()[3]
g <- construct_im_graph(n1,n2,0.1,0.1)

cat(paste("Construction took ", proc.time()[3] - time, "seconds\n"))

expected_visits <- rep(0, vertices_length(g))
time <- proc.time()[3]
ctx <- distribution_context(g,1000)
while (distribution_context_state(ctx)$time <= 1.5) {
    distribution_context_step(ctx)
}

cat(paste("Finding stopping time took ", proc.time()[3] - time, "seconds\n"))


algorithm_expectation <- matrix(nrow=n1+1,ncol=n2+1)

time <- proc.time()[3]
k <- expectation(g)

cat(paste("Moment graph took ", proc.time()[3] - time, "seconds\n"))

time <- proc.time()[3]
for (i in 0:n1) {
  for (j in 0:n2) {
    algorithm_expectation[i+1, j+1]<- expectation(g, rewards_at(g, i,j,n1,n2))
    }
}

cat(paste("Expectations took ", proc.time()[3] - time, "seconds\n"))
algorithm_expectation

Construction took  8.26 seconds
Finding stopping time took  147.201 seconds
Moment graph took  684.106 seconds
Expectations took  59.935 seconds


0.0000000,2.13609080,1.11315340,0.78181022,0.63148324,0.56624840,0.57205493,0.72144128,2.8280090
2.1360908,0.07749005,0.05854808,0.05465698,0.05573244,0.06099544,0.07349038,0.10893591,0.5399685
1.1131534,0.05854808,0.03817934,0.03280239,0.03188839,0.03395269,0.04028362,0.05931562,0.3234753
0.7818102,0.05465698,0.03280239,0.02632106,0.02426083,0.02476811,0.02838357,0.04054851,0.2399767
0.6314832,0.05573244,0.03188839,0.02426083,0.02124944,0.02067761,0.02261989,0.03079193,0.1959471
0.5662484,0.06099544,0.03395269,0.02476811,0.02067761,0.01910199,0.01973611,0.02510184,0.1692997
0.5720549,0.07349038,0.04028362,0.02838357,0.02261989,0.01973611,0.01900748,0.02199899,0.1522455
0.7214413,0.10893591,0.05931562,0.04054851,0.03079193,0.02510184,0.02199899,0.02208418,0.1420733
2.8280090,0.53996845,0.32347530,0.23997669,0.19594711,0.16929970,0.15224550,0.14207334,0.0000000


In [5]:
# Comparing results to verify implementation/numerical accuracy
n1 <- 4
n2 <- 4

g <- construct_im_graph(n1,n2,0.1,0.1)

expected_visits <- rep(0, vertices_length(g))
ctx <- distribution_context(g,1000)
while (distribution_context_state(ctx)$cdf < 0.9999) {
  distribution_context_step(ctx)
}

expected_visits <- distribution_context_accumulated_visiting_time(ctx)

distribution_expectation <- matrix(nrow=n1+1,ncol=n2+1)
algorithm_expectation <- matrix(nrow=n1+1,ncol=n2+1)
matrix_expectation <- matrix(nrow=n1+1,ncol=n2+1)
simulation_expectation <- matrix(nrow=n1+1,ncol=n2+1)
PH <- graph_as_matrix(g)
U <- solve(-PH$SIM)
set.seed(1234)

for (i in 0:n1) {
  for (j in 0:n2) {
    distribution_expectation[i+1,j+1] <- sum(expected_visits * rewards_at(g, i,j,n1,n2))
    algorithm_expectation[i+1, j+1]<- expectation(g, rewards_at(g, i,j,n1,n2))
    simulation_expectation[i+1, j+1]<- mean(rph(10000, g, rewards_at(g, i,j,n1,n2)))
    matrix_expectation[i+1,j+1] <-PH$IPV %*% U%*%diag(PH$states[,(matrix_index(i,j,0,n1,n2)+1)]+PH$states[,(matrix_index(i,j,1,n1,n2)+1)])%*%rep(1,length(PH$IPV))
  }
}

sum(abs(matrix_expectation - algorithm_expectation))
sum(abs(matrix_expectation - simulation_expectation))
sum(abs(matrix_expectation - distribution_expectation))

[1] 1.111611e-14

[1] 0.3662765

[1] 0.001238273